In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd

In [2]:
# 
model_root_and_name_tuple_list = [("dbnsfp", "sift"), ("dbnsfp", "polyphen2_HVAR"), ("dbnsfp", "cadd"), ("dbnsfp", "mvp"), ("dbnsfp", "metarnn"), ("dbnsfp", "revel"),
                                  ("tape_rao", "unirep"), ("tape_rao", "protbert"), ("sequnet_dunham", "sequnet"), 
                                  ("esm_rives", "esm1b_t33_650M_UR50S"), ("esm_rives", "esm1v_t33_650M_UR90S"), ("esm_rives", "esm2_t33_650M_UR50D"),
                                  ("bioembeddings_dallago", "plus_rnn"), 
                                  ("bioembeddings_dallago", "prottrans_bert_bfd"), ("bioembeddings_dallago", "prottrans_albert_bfd"),
                                  ("bioembeddings_dallago", "prottrans_xlnet_uniref100"), 
                                  ("bioembeddings_dallago", "prottrans_t5_bfd"), ("bioembeddings_dallago", "prottrans_t5_uniref50"), ("bioembeddings_dallago", "prottrans_t5_xl_u50")]

### CDD Conservation dataframe

In [12]:
conservation_df = pd.read_csv(home_dir+"data/cdd_conservationTable_18kHumanProts.csv.gzip", compression='gzip', comment='#')
print(conservation_df.shape)
print(conservation_df.columns)
conservation_df

(4104012, 36)
Index(['qNo', 'NPid', 'accession', 'que', 'sub', 'ali', 'qPos', 'sPos', 'aPos',
       'qPos_', 'sPos_', 'bitscore', 'A', 'D', 'C', 'E', 'F', 'G', 'H', 'I',
       'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'cons',
       'entropy', 'qcov', 'CScore'],
      dtype='object')


,qNo,NPid,accession,que,sub,ali,qPos,sPos,aPos,qPos_,...,R,S,T,V,W,Y,cons,entropy,qcov,CScore
0,1,NP_064505.1,pfam18400,S,A,|,57,1,1,57,...,0.000000,0.303030,0.000000,0.000000,0.0,0.000000,X,0.492204,0.715113,0.507796
1,1,NP_064505.1,pfam18400,T,T,|,58,2,2,58,...,0.000000,0.030303,0.555556,0.000000,0.0,0.000000,X,0.288488,0.715113,0.711512
2,1,NP_064505.1,pfam18400,P,P,|,59,3,3,59,...,0.000000,0.101010,0.000000,0.000000,0.0,0.000000,P,0.125695,0.715113,0.874305
3,1,NP_064505.1,pfam18400,L,L,|,60,4,4,60,...,0.000000,0.000000,0.000000,0.050505,0.0,0.161616,X,0.479242,0.715113,0.520758
4,1,NP_064505.1,pfam18400,L,L,|,61,5,5,61,...,0.000000,0.010101,0.010101,0.050505,0.0,0.020202,L,0.352476,0.715113,0.647524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4104007,961,NP_001091888.1,pfam05831,T,M,|,107,103,108,107,...,0.041667,0.000000,0.041667,0.125000,0.0,0.000000,X,0.554522,0.905983,0.445478
4104008,961,NP_001091888.1,pfam05831,P,P,|,108,104,109,108,...,0.000000,0.041667,0.000000,0.000000,0.0,0.000000,P,0.308799,0.905983,0.691201
4104009,961,NP_001091888.1,pfam05831,E,E,,109,105,110,109,...,0.041667,0.000000,0.000000,0.000000,0.0,0.000000,E,0.278218,0.905983,0.721782
4104010,961,NP_001091888.1,pfam05831,E,A,|,110,106,111,110,...,0.000000,0.000000,0.041667,0.000000,0.0,0.000000,X,0.315743,0.905983,0.684257


In [13]:
def merge_conservation_df(df:pd.DataFrame):
    # merge conservation_df with the input df
    temp_conservation_df = conservation_df[["NPid", "qPos", "CScore"]].drop_duplicates(keep="first")
    merged_df = pd.merge(df, temp_conservation_df, how="left", left_on=["prot_acc_version", "prot_pos"], right_on=["NPid", "qPos"])
    columns = list(df.columns)
    columns.append("CScore")
    merged_df = merged_df[columns]
    merged_df = merged_df.rename(columns={"CScore": "conservation_pred"})
    return merged_df

### Merging pathogenicity analysis predictions

In [7]:
pathogenicity_type = "likely_pathogenic" # pathogenic, likely_pathogenic

from models.aa_common.data_loader import get_pathogenicity_analysis_SNVs
list_of_variants_df = get_pathogenicity_analysis_SNVs(home_dir=home_dir, pathogenicity_type=pathogenicity_type)


Log: Loading combined fasta iterator ...
likely_pathogenic raw data: (6668, 12)
Index(['clinvar_id', 'gene_symbol', 'gene_id', 'chrom_acc_version',
       'chrom_pos', 'ref_allele', 'alt_allele', 'prot_acc_version', 'prot_pos',
       'wt', 'mut', 'class'],
      dtype='object')

Log: excluding variants corresponding to proteins having seq-len>1022 ...

Log: Loading combined fasta iterator ...
#-protein sequences (seq-len<=1022): 1077
(4168, 12)

Log: Loading population freq variants dataset ...
Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq'],
      dtype='object')
0 (8336, 10)
1 (8336, 10)
2 (8336, 10)
3 (8336, 10)
4 (8336, 10)
5 (8336, 10)
6 (8336, 10)
7 (8336, 10)
8 (8336, 10)
9 (8336, 10)


In [8]:
for analysis_no, patho_variants_df in enumerate(list_of_variants_df):
    print(analysis_no)
    merged_df = patho_variants_df.copy(deep=True)
    merge_col_list = list(patho_variants_df.columns)
    
    for i, (model_root, model_name) in enumerate(model_root_and_name_tuple_list):    
        models_pred_df = pd.read_csv(home_dir+f"models/{model_root}/outputs/{model_name}/{pathogenicity_type}/{str(analysis_no)}.csv", sep="\t")
        print(model_root, model_name, models_pred_df.shape)
        merged_df = pd.merge(left=merged_df, right=models_pred_df, how="left", on=merge_col_list)
        merged_df = merged_df.rename(columns={"pred": f"{model_name}_pred"})
        print(merged_df.shape)
        # if i==5: break
    
    # merged_df = merge_conservation_df(merged_df)  
    # print("conservation_score")  
    print(merged_df.shape)
    merged_df.to_csv(home_dir+f"models/aa_common/merged_predictions/{pathogenicity_type}_analysis/{str(analysis_no)}.csv", sep="\t", index=False, header=True)
    # break

0
dbnsfp sift (8336, 11)
(8336, 11)
dbnsfp polyphen2_HVAR (8336, 11)
(8336, 12)
dbnsfp cadd (8336, 11)
(8336, 13)
dbnsfp mvp (8336, 11)
(8336, 14)
dbnsfp metarnn (8336, 11)
(8336, 15)
dbnsfp revel (8336, 11)
(8336, 16)
tape_rao unirep (8336, 11)
(8336, 17)
tape_rao protbert (8336, 11)
(8336, 18)
sequnet_dunham sequnet (8327, 11)
(8336, 19)
esm_rives esm1b_t33_650M_UR50S (8336, 11)
(8336, 20)
esm_rives esm1v_t33_650M_UR90S (8336, 11)
(8336, 21)
esm_rives esm2_t33_650M_UR50D (8336, 11)
(8336, 22)
bioembeddings_dallago plus_rnn (8336, 11)
(8336, 23)
bioembeddings_dallago prottrans_bert_bfd (8336, 11)
(8336, 24)
bioembeddings_dallago prottrans_albert_bfd (8336, 11)
(8336, 25)
bioembeddings_dallago prottrans_xlnet_uniref100 (8336, 11)
(8336, 26)
bioembeddings_dallago prottrans_t5_bfd (8336, 11)
(8336, 27)
bioembeddings_dallago prottrans_t5_uniref50 (8336, 11)
(8336, 28)
bioembeddings_dallago prottrans_t5_xl_u50 (8336, 11)
(8336, 29)
conservation_score
(8336, 30)
1
dbnsfp sift (8336, 11)
(83

### Merging population frequency predictions

In [5]:
from models.aa_common.data_loader import get_population_freq_SNVs
popu_freq_variants_df = get_population_freq_SNVs(home_dir=home_dir)
popu_freq_variants_df


Log: Loading data ...
(407591, 13)
Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq'],
      dtype='object')
NP_001840.3       139
NP_001277137.1    138
NP_001291317.1    136
NP_071375.1       135
NP_005996.2       126
                 ... 
NP_219479.2         1
NP_001259000.1      1
NP_006348.1         1
NP_002361.1         1
NP_057387.1         1
Name: prot_acc_version, Length: 15871, dtype: int64
Common: (18167), rare: (28622), ultra-rare: (314013), sampled-singletons: (46789) and total: 407591


,snp_id,chrom_acc_version,chrom_pos,ref_allele,alt_allele,prot_acc_version,prot_pos,wt,mut,wt_population,mut_poulation,wt_freq,mt_freq,class
0,rs41288789,NC_000002.12,26944749,G,A,NP_064519.2,512,A,T,218102,10302,0.954896,0.045104,Common
1,rs34603401,NC_000001.11,9245386,A,C,NP_004276.2,151,D,A,225918,32086,0.875638,0.124362,Common
2,rs115026899,NC_000001.11,210683774,C,A,NP_758872.1,826,G,V,211392,6706,0.969252,0.030748,Common
3,rs17260829,NC_000006.12,122451974,T,C,NP_065806.1,225,S,G,301772,13333,0.957687,0.042313,Common
4,rs1438318937,NC_000019.10,54575019,A,G,NP_001124389.2,214,E,G,11862,5489,0.683649,0.316351,Common
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407586,rs1311034485,NC_000013.11,71489095,G,C,NP_542937.3,542,L,V,35428,1,0.999972,0.000028,Singleton
407587,rs751186776,NC_000019.10,12896958,G,A,NP_000150.1,301,V,M,14528,1,0.999931,0.000069,Singleton
407588,rs200406718,NC_000017.11,76387303,G,A,NP_001136073.1,291,R,Q,41778,1,0.999976,0.000024,Singleton
407589,rs753278261,NC_000010.11,80432091,A,G,NP_115709.3,228,K,E,14050,1,0.999929,0.000071,Singleton


In [6]:
task = "popu_freq"
merged_df = popu_freq_variants_df.copy(deep=True)
merge_col_list = list(popu_freq_variants_df.columns)
merge_col_list.remove("class")

for i, (model_root, model_name) in enumerate(model_root_and_name_tuple_list):
    models_pred_df = pd.read_csv(home_dir+f"models/{model_root}/outputs/{model_name}/{task}/preds_{model_name}.csv", sep="\t")
    print(model_root, model_name, models_pred_df.shape)
    merged_df = pd.merge(left=merged_df, right=models_pred_df, how="left", on=merge_col_list)
    merged_df = merged_df.rename(columns={"pred": f"{model_name}_pred"})
    merged_df = merged_df.drop_duplicates(keep="first")
    print(merged_df.shape)
    # if i==3: break

# merged_df = merge_conservation_df(merged_df)  
# print("conservation_score")  
print(merged_df.shape)
merged_df.to_csv(home_dir+f"models/aa_common/merged_predictions/{task}_analysis_1.csv", sep="\t", index=False, header=True)

dbnsfp sift (407591, 14)
(407591, 15)
dbnsfp polyphen2_HVAR (407591, 14)
(407591, 16)
dbnsfp cadd (407591, 14)
(407591, 17)
dbnsfp mvp (407591, 14)
(407591, 18)
dbnsfp metarnn (407591, 14)
(407591, 19)
dbnsfp revel (407591, 14)
(407591, 20)
tape_rao unirep (407591, 14)
(407591, 21)
tape_rao protbert (407591, 14)
(407591, 22)
sequnet_dunham sequnet (407098, 14)
(407591, 23)
esm_rives esm1b_t33_650M_UR50S (407591, 14)
(407591, 24)
esm_rives esm1v_t33_650M_UR90S (407591, 14)
(407591, 25)
esm_rives esm2_t33_650M_UR50D (407591, 14)
(407591, 26)
bioembeddings_dallago plus_rnn (407591, 14)
(407591, 27)
bioembeddings_dallago prottrans_bert_bfd (407591, 14)
(407591, 28)
bioembeddings_dallago prottrans_albert_bfd (407591, 14)
(407591, 29)
bioembeddings_dallago prottrans_xlnet_uniref100 (407591, 14)
(407591, 30)
bioembeddings_dallago prottrans_t5_bfd (407591, 14)
(407591, 31)
bioembeddings_dallago prottrans_t5_uniref50 (407591, 14)
(407591, 32)
bioembeddings_dallago prottrans_t5_xl_u50 (407591, 1

In [8]:
n_commnon = merged_df[merged_df["class"]=="Common"].shape[0]
n_rare = merged_df[merged_df["class"]=="Rare"].shape[0]
n_ultra_rare = merged_df[merged_df["class"]=="Ultra-rare"].shape[0]
n_singleton = merged_df[merged_df["class"]=="Singleton"].shape[0]
print(f"Common: ({n_commnon}), rare: ({n_rare}), ultra-rare: ({n_ultra_rare}) sampled-singletons: ({n_singleton}) and total: {merged_df.shape[0]}")

Common: (18167), rare: (28622), ultra-rare: (314013) sampled-singletons: (46789) and total: 407591


### Merging protein mutation dataset (PMD) predictions

In [11]:
from models.aa_common.data_loader import get_pmd_analysis_dataset
pmd_variants_df = get_pmd_analysis_dataset(home_dir=home_dir)
pmd_variants_df["functional_effect"].value_counts()


Log: Loading Protein Mutation Dataset (PMD) ...
(55465, 15)
Index(['mut_id', 'pmd_id', 'protein', 'mut_PMD', 'mut_real', 'function',
       'taxid', 'function_e', 'function_e2', 'functional_effect', 'seq',
       'protein_id', 'wt', 'mut', 'prot_pos'],
      dtype='object')

Log: excluding variants corresponding to proteins having seq-len>1022 ...

Log: Loading combined fasta iterator ...
#-protein sequences (seq-len<=1022): 7677
(51047, 15)


Knock-out    25116
No effect    13297
Effect       12634
Name: functional_effect, dtype: int64

In [9]:
task = "pmd_analysis"
merged_df = pmd_variants_df.copy(deep=True)
merge_col_list = list(pmd_variants_df.columns)

for i, (model_root, model_name) in enumerate(model_root_and_name_tuple_list):
    models_pred_df = pd.read_csv(home_dir+f"models/{model_root}/outputs/{model_name}/{task}/preds_{model_name}_(mt-wt).csv", sep=",")
    print(model_root, model_name, models_pred_df.shape)
    merged_df = pd.merge(left=merged_df, right=models_pred_df, how="left", on=merge_col_list)
    merged_df = merged_df.rename(columns={"pred": f"{model_name}_pred"})
    print(merged_df.shape)
    # if i==5: break
    
merged_df.to_csv(home_dir+f"models/aa_common/merged_predictions/{task}.csv", sep="\t", index=False, header=True)

tape_rao unirep (51047, 16)
(51047, 16)
tape_rao protbert (51047, 16)
(51047, 17)
sequnet_dunham sequnet (50994, 16)
(51047, 18)
esm_rives esm1b_t33_650M_UR50S (51047, 16)
(51047, 19)
esm_rives esm1v_t33_650M_UR90S (51047, 16)
(51047, 20)
esm_rives esm2_t33_650M_UR50D (51047, 16)
(51047, 21)
bioembeddings_dallago plus_rnn (51047, 16)
(51047, 22)
bioembeddings_dallago prottrans_bert_bfd (51047, 16)
(51047, 23)
bioembeddings_dallago prottrans_albert_bfd (51047, 16)
(51047, 24)
bioembeddings_dallago prottrans_xlnet_uniref100 (51047, 16)
(51047, 25)
bioembeddings_dallago prottrans_t5_bfd (51047, 16)
(51047, 26)
bioembeddings_dallago prottrans_t5_uniref50 (51047, 16)
(51047, 27)
bioembeddings_dallago prottrans_t5_xl_u50 (51047, 16)
(51047, 28)
